In [14]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
attrition_df = pd.read_csv('HR-Employee-Attrition.csv')
attrition_df

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


In [15]:
# Generate our categorical variable list
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()
attrition_cat

['Attrition',
 'BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'Over18',
 'OverTime']

In [16]:
# Check the number of unique values in each column
attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

In [17]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [18]:
# Merge one-hot encoded features and drop the originals
attrition_df = attrition_df.merge(encode_df,left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat,1)
attrition_df.head()

/Users/emilymaynard/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [19]:
# Split our preprocessed data into our features and target arrays
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes","Attrition_No"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

/Users/emilymaynard/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [21]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8)                 448       
                                                                 
 dense_4 (Dense)             (None, 5)                 45        
                                                                 
 dense_5 (Dense)             (None, 1)                 6         
                                                                 
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


# To create checkpoints on each epoch

In [22]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [23]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
25/35 [====================>.........] - ETA: 0s - loss: 0.5947 - accuracy: 0.8075 
Epoch 1: saving model to checkpoints/weights.01.hdf5
35/35 [==============================] - 1s 4ms/step - loss: 0.5885 - accuracy: 0.8094
Epoch 2/100
20/35 [================>.............] - ETA: 0s - loss: 0.5403 - accuracy: 0.8344
Epoch 2: saving model to checkpoints/weights.02.hdf5
35/35 [==============================] - 0s 3ms/step - loss: 0.5366 - accuracy: 0.8276
Epoch 3/100
34/35 [============================>.] - ETA: 0s - loss: 0.4900 - accuracy: 0.8290
Epoch 3: saving model to checkpoints/weights.03.hdf5
35/35 [==============================] - 0s 4ms/step - loss: 0.4941 - accuracy: 0.8276
Epoch 4/100
26/35 [=====================>........] - ETA: 0s - loss: 0.4741 - accuracy: 0.8257
Epoch 4: saving model to checkpoints/weights.04.hdf5
35/35 [==============================] - 0s 4ms/step - loss: 0.4626 - accuracy: 0.8276
Epoch 5/100
32/35 [==========================>...] - ETA: 0

35/35 [==============================] - 0s 3ms/step - loss: 0.2636 - accuracy: 0.8984
Epoch 36/100
33/35 [===========================>..] - ETA: 0s - loss: 0.2660 - accuracy: 0.8949
Epoch 36: saving model to checkpoints/weights.36.hdf5
35/35 [==============================] - 0s 4ms/step - loss: 0.2609 - accuracy: 0.8975
Epoch 37/100
23/35 [==================>...........] - ETA: 0s - loss: 0.2585 - accuracy: 0.9035
Epoch 37: saving model to checkpoints/weights.37.hdf5
35/35 [==============================] - 0s 3ms/step - loss: 0.2584 - accuracy: 0.8993
Epoch 38/100
29/35 [=======================>......] - ETA: 0s - loss: 0.2464 - accuracy: 0.8987
Epoch 38: saving model to checkpoints/weights.38.hdf5
35/35 [==============================] - 0s 4ms/step - loss: 0.2559 - accuracy: 0.8993
Epoch 39/100
22/35 [=================>............] - ETA: 0s - loss: 0.2532 - accuracy: 0.8977
Epoch 39: saving model to checkpoints/weights.39.hdf5
35/35 [==============================] - 0s 3ms/step

28/35 [=======================>......] - ETA: 0s - loss: 0.1967 - accuracy: 0.9319
Epoch 70: saving model to checkpoints/weights.70.hdf5
35/35 [==============================] - 0s 4ms/step - loss: 0.1904 - accuracy: 0.9383
Epoch 71/100
28/35 [=======================>......] - ETA: 0s - loss: 0.1920 - accuracy: 0.9442
Epoch 71: saving model to checkpoints/weights.71.hdf5
35/35 [==============================] - 0s 11ms/step - loss: 0.1880 - accuracy: 0.9419
Epoch 72/100
28/35 [=======================>......] - ETA: 0s - loss: 0.1793 - accuracy: 0.9431
Epoch 72: saving model to checkpoints/weights.72.hdf5
35/35 [==============================] - 0s 5ms/step - loss: 0.1868 - accuracy: 0.9392
Epoch 73/100
26/35 [=====================>........] - ETA: 0s - loss: 0.1817 - accuracy: 0.9423
Epoch 73: saving model to checkpoints/weights.73.hdf5
35/35 [==============================] - 0s 5ms/step - loss: 0.1852 - accuracy: 0.9419
Epoch 74/100
28/35 [=======================>......] - ETA: 0s - 

In [24]:
#this time rerun the code using the checkpoints
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.5469 - accuracy: 0.8315 - 208ms/epoch - 17ms/step
Loss: 0.5468760132789612, Accuracy: 0.83152174949646


# To export the trained model

In [25]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")

In [ ]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

In [ ]:
# Evaluate the completed model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

## Run the code without checkpoints

In [9]:
# Here we ran the model without the checkppoints
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=500)

Epoch 1/500
35/35 [==============================] - 0s 3ms/step - loss: 0.5137 - accuracy: 0.8158
Epoch 2/500
35/35 [==============================] - 0s 4ms/step - loss: 0.6557 - accuracy: 0.7931
Epoch 3/500
35/35 [==============================] - 0s 2ms/step - loss: 1.8459 - accuracy: 0.7287
Epoch 4/500
35/35 [==============================] - 0s 3ms/step - loss: 0.6647 - accuracy: 0.7995
Epoch 5/500
35/35 [==============================] - 0s 3ms/step - loss: 0.5498 - accuracy: 0.8040
Epoch 6/500
35/35 [==============================] - 0s 3ms/step - loss: 0.7979 - accuracy: 0.7722
Epoch 7/500
35/35 [==============================] - 0s 4ms/step - loss: 0.9121 - accuracy: 0.7795
Epoch 8/500
35/35 [==============================] - 0s 3ms/step - loss: 0.8917 - accuracy: 0.7604
Epoch 9/500
35/35 [==============================] - 0s 2ms/step - loss: 0.7238 - accuracy: 0.7831
Epoch 10/500
35/35 [==============================] - 0s 3ms/step - loss: 0.6415 - accuracy: 0.7804
Epoch 11/

35/35 [==============================] - 0s 3ms/step - loss: 0.4296 - accuracy: 0.8212
Epoch 84/500
35/35 [==============================] - 0s 2ms/step - loss: 0.8161 - accuracy: 0.7568
Epoch 85/500
35/35 [==============================] - 0s 3ms/step - loss: 0.9386 - accuracy: 0.7668
Epoch 86/500
35/35 [==============================] - 0s 2ms/step - loss: 1.4304 - accuracy: 0.7432
Epoch 87/500
35/35 [==============================] - 0s 2ms/step - loss: 1.2688 - accuracy: 0.7668
Epoch 88/500
35/35 [==============================] - 0s 2ms/step - loss: 0.5469 - accuracy: 0.8067
Epoch 89/500
35/35 [==============================] - 0s 3ms/step - loss: 0.5119 - accuracy: 0.8076
Epoch 90/500
35/35 [==============================] - 0s 2ms/step - loss: 0.6248 - accuracy: 0.7877
Epoch 91/500
35/35 [==============================] - 0s 2ms/step - loss: 0.6789 - accuracy: 0.7868
Epoch 92/500
35/35 [==============================] - 0s 3ms/step - loss: 0.8291 - accuracy: 0.7768
Epoch 93/500


35/35 [==============================] - 0s 2ms/step - loss: 0.5409 - accuracy: 0.8094
Epoch 165/500
35/35 [==============================] - 0s 2ms/step - loss: 0.5111 - accuracy: 0.8367
Epoch 166/500
35/35 [==============================] - 0s 2ms/step - loss: 1.5389 - accuracy: 0.7477
Epoch 167/500
35/35 [==============================] - 0s 2ms/step - loss: 0.6160 - accuracy: 0.8140
Epoch 168/500
35/35 [==============================] - 0s 2ms/step - loss: 0.4442 - accuracy: 0.8367
Epoch 169/500
35/35 [==============================] - 0s 2ms/step - loss: 0.6672 - accuracy: 0.7976
Epoch 170/500
35/35 [==============================] - 0s 2ms/step - loss: 0.6998 - accuracy: 0.7822
Epoch 171/500
35/35 [==============================] - 0s 2ms/step - loss: 0.7472 - accuracy: 0.8040
Epoch 172/500
35/35 [==============================] - 0s 2ms/step - loss: 0.7798 - accuracy: 0.7904
Epoch 173/500
35/35 [==============================] - 0s 2ms/step - loss: 0.7522 - accuracy: 0.7958
Epoc

35/35 [==============================] - 0s 2ms/step - loss: 0.5839 - accuracy: 0.8158
Epoch 246/500
35/35 [==============================] - 0s 2ms/step - loss: 0.7567 - accuracy: 0.7813
Epoch 247/500
35/35 [==============================] - 0s 2ms/step - loss: 0.4918 - accuracy: 0.8149
Epoch 248/500
35/35 [==============================] - 0s 3ms/step - loss: 0.5500 - accuracy: 0.8158
Epoch 249/500
35/35 [==============================] - 0s 2ms/step - loss: 0.4180 - accuracy: 0.8412
Epoch 250/500
35/35 [==============================] - 0s 2ms/step - loss: 0.5611 - accuracy: 0.8258
Epoch 251/500
35/35 [==============================] - 0s 3ms/step - loss: 0.3865 - accuracy: 0.8466
Epoch 252/500
35/35 [==============================] - 0s 3ms/step - loss: 0.4167 - accuracy: 0.8385
Epoch 253/500
35/35 [==============================] - 0s 2ms/step - loss: 0.4062 - accuracy: 0.8448
Epoch 254/500
35/35 [==============================] - 0s 3ms/step - loss: 0.4649 - accuracy: 0.8194
Epoc

35/35 [==============================] - 0s 3ms/step - loss: 0.3917 - accuracy: 0.8466
Epoch 327/500
35/35 [==============================] - 0s 2ms/step - loss: 0.5344 - accuracy: 0.7995
Epoch 328/500
35/35 [==============================] - 0s 2ms/step - loss: 0.6425 - accuracy: 0.8031
Epoch 329/500
35/35 [==============================] - 0s 2ms/step - loss: 0.7095 - accuracy: 0.7813
Epoch 330/500
35/35 [==============================] - 0s 2ms/step - loss: 0.8373 - accuracy: 0.7886
Epoch 331/500
35/35 [==============================] - 0s 2ms/step - loss: 0.6969 - accuracy: 0.7868
Epoch 332/500
35/35 [==============================] - 0s 2ms/step - loss: 0.4583 - accuracy: 0.8258
Epoch 333/500
35/35 [==============================] - 0s 3ms/step - loss: 0.4509 - accuracy: 0.8367
Epoch 334/500
35/35 [==============================] - 0s 2ms/step - loss: 0.4230 - accuracy: 0.8367
Epoch 335/500
35/35 [==============================] - 0s 2ms/step - loss: 0.4869 - accuracy: 0.8258
Epoc

35/35 [==============================] - 0s 3ms/step - loss: 0.4704 - accuracy: 0.8267
Epoch 408/500
35/35 [==============================] - 0s 3ms/step - loss: 0.4251 - accuracy: 0.8303
Epoch 409/500
35/35 [==============================] - 0s 2ms/step - loss: 0.4211 - accuracy: 0.8348
Epoch 410/500
35/35 [==============================] - 0s 3ms/step - loss: 0.5886 - accuracy: 0.8122
Epoch 411/500
35/35 [==============================] - 0s 2ms/step - loss: 0.6445 - accuracy: 0.8103
Epoch 412/500
35/35 [==============================] - 0s 2ms/step - loss: 1.2803 - accuracy: 0.7559
Epoch 413/500
35/35 [==============================] - 0s 2ms/step - loss: 0.6551 - accuracy: 0.7904
Epoch 414/500
35/35 [==============================] - 0s 2ms/step - loss: 0.4418 - accuracy: 0.8521
Epoch 415/500
35/35 [==============================] - 0s 2ms/step - loss: 0.4884 - accuracy: 0.8330
Epoch 416/500
35/35 [==============================] - 0s 2ms/step - loss: 0.6793 - accuracy: 0.8067
Epoc

35/35 [==============================] - 0s 3ms/step - loss: 0.3994 - accuracy: 0.8439
Epoch 489/500
35/35 [==============================] - 0s 2ms/step - loss: 0.4865 - accuracy: 0.8276
Epoch 490/500
35/35 [==============================] - 0s 2ms/step - loss: 0.3417 - accuracy: 0.8621
Epoch 491/500
35/35 [==============================] - 0s 2ms/step - loss: 0.4581 - accuracy: 0.8376
Epoch 492/500
35/35 [==============================] - 0s 2ms/step - loss: 0.5348 - accuracy: 0.8203
Epoch 493/500
35/35 [==============================] - 0s 2ms/step - loss: 0.7174 - accuracy: 0.8103
Epoch 494/500
35/35 [==============================] - 0s 3ms/step - loss: 0.5328 - accuracy: 0.8221
Epoch 495/500
35/35 [==============================] - 0s 2ms/step - loss: 0.4050 - accuracy: 0.8475
Epoch 496/500
35/35 [==============================] - 0s 2ms/step - loss: 0.4014 - accuracy: 0.8403
Epoch 497/500
35/35 [==============================] - 0s 2ms/step - loss: 0.4370 - accuracy: 0.8321
Epoc

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 1.0352 - accuracy: 0.8342 - 31ms/epoch - 3ms/step
Loss: 1.0351966619491577, Accuracy: 0.83423912525177
